In [2]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Opening SQL Data

In [3]:
connection = sqlite3.connect('sakila.db')

In [4]:
query = '''
SELECT address.address, city.city, country.country
FROM address
JOIN city ON address.city_id = city.city_id
JOIN country ON city.country_id = country.country_id
'''

#### Creating DF + data cleaning

In [5]:
df = pd.read_sql_query(query, connection)

In [6]:
connection.close()

In [7]:
df

,address,city,country
0,47 MySakila Drive,Lethbridge,Canada
1,28 MySQL Boulevard,Woodridge,Australia
2,23 Workhaven Lane,Lethbridge,Canada
3,1411 Lillydale Drive,Woodridge,Australia
4,1913 Hanoi Way,Sasebo,Japan
...,...,...,...
598,844 Bucuresti Place,Jinzhou,China
599,1101 Bucuresti Boulevard,Patras,Greece
600,1103 Quilmes Boulevard,Sullana,Peru
601,1331 Usak Boulevard,Lausanne,Switzerland


In [8]:
poland_locations = df.loc[df['country'] == 'Poland'].copy()

In [9]:
poland_locations

,address,city,country
131,1050 Garden Grove Avenue,Jastrzebie-Zdrj,Poland
273,906 Goinia Way,Kalisz,Poland
321,1769 Iwaki Lane,Bydgoszcz,Poland
383,1642 Charlotte Amalie Drive,Tychy,Poland
504,414 Mandaluyong Street,Lublin,Poland
561,505 Madiun Boulevard,Wroclaw,Poland
574,1501 Pangkal Pinang Avenue,Plock,Poland
575,1405 Hagonoy Avenue,Czestochowa,Poland


In [10]:
poland_locations.loc[poland_locations['city'] == "Jastrzebie-Zdrj", 'city'] = "Jastrzębie-Zdroj"

In [11]:
poland_locations

,address,city,country
131,1050 Garden Grove Avenue,Jastrzębie-Zdroj,Poland
273,906 Goinia Way,Kalisz,Poland
321,1769 Iwaki Lane,Bydgoszcz,Poland
383,1642 Charlotte Amalie Drive,Tychy,Poland
504,414 Mandaluyong Street,Lublin,Poland
561,505 Madiun Boulevard,Wroclaw,Poland
574,1501 Pangkal Pinang Avenue,Plock,Poland
575,1405 Hagonoy Avenue,Czestochowa,Poland


#### Decoding coordinates with Nominatim

In [12]:
from geopy.geocoders import Nominatim

In [13]:
geolocator = Nominatim(user_agent="geoapiExercises")

In [30]:
def update_coordinates(data):
    for index, city in data['city'].items():
        location = geolocator.geocode(f"{city}, Poland", timeout=5)
        if location:
            data.loc[index, 'latitude'] = location.latitude
            data.loc[index, 'longitude'] = location.longitude
        else:
            print(f"Geocoding failed for city: {city}")

    return data

In [31]:
poland_locations = update_coordinates(poland_locations.copy())

In [32]:
print(poland_locations[['city', 'latitude', 'longitude']])

                 city   latitude  longitude
131  Jastrzębie-Zdroj  49.960679  18.599372
273            Kalisz  51.747365  18.079590
321         Bydgoszcz  53.129746  18.029370
383             Tychy  50.114397  18.996593
504            Lublin  51.250559  22.570102
561           Wroclaw  51.108978  17.032669
574             Plock  52.546452  19.700861
575       Czestochowa  50.809000  19.124409


#### Showing locations on the map with Folium

In [25]:
import folium

In [26]:
mean_latitude = poland_locations['latitude'].mean()
mean_longitude = poland_locations['longitude'].mean()

In [27]:
map = folium.Map(location=[mean_latitude, mean_longitude], zoom_start=7, title='Sakila locations in Poland')

In [28]:
for _, row in poland_locations.iterrows():
    folium.Marker([row['latitude'], row['longitude']], tooltip=row['city']).add_to(map)

In [29]:
map